In [1]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import time


2022-01-26 01:35:45.757647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lordcocoro2004/miniconda3/envs/rcnn_2/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-26 01:35:45.757684: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
#with strategy.scope():
vggmodel1 = VGG16(weights='imagenet', include_top=True)
vggmodel2 = VGG16(weights='imagenet', include_top=True)
vggmodel3 = VGG16(weights='imagenet', include_top=True)
vggmodel4 = VGG16(weights='imagenet', include_top=True)
vggmodel5 = VGG16(weights='imagenet', include_top=True)


2022-01-26 01:35:48.057876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lordcocoro2004/miniconda3/envs/rcnn_2/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-26 01:35:48.057915: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-26 01:35:48.057934: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t1v-n-639a9f36-w-0): /proc/driver/nvidia/version does not exist
2022-01-26 01:35:48.058145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [3]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

X1= vggmodel1.layers[-2].output
X2= vggmodel2.layers[-2].output
X3= vggmodel3.layers[-2].output
X4= vggmodel4.layers[-2].output
X5= vggmodel5.layers[-2].output

predictions_g= Dense(2, activation="softmax")(X1)
predictions_m = Dense(2, activation="softmax")(X2)
predictions_p = Dense(2, activation="softmax")(X3)
predictions_t = Dense(2, activation="softmax")(X4)
predictions_z = Dense(2, activation="softmax")(X5)

model_gas = Model(vggmodel1.input, predictions_g)
model_mineral = Model(vggmodel2.input, predictions_m)
model_protos = Model(vggmodel3.input, predictions_p)
model_terran = Model(vggmodel4.input, predictions_t)
model_zerg = Model(vggmodel5.input, predictions_z)

opt = Adam(lr=0.0001)
model_gas.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_mineral.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_protos.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_terran.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_zerg.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])

model_gas.summary()
model_mineral.summary()
model_protos.summary()
model_terran.summary()
model_zerg.summary()

model_gas.load_weights('/home/lordcocoro2004/rcnn/ieeercnn_vgg16_1gas.h5')
model_mineral.load_weights('/home/lordcocoro2004/rcnn/ieeercnn_vgg16_1mineral.h5')
model_protos.load_weights('/home/lordcocoro2004/rcnn/ieeercnn_vgg16_1protosbase.h5')
model_terran.load_weights('/home/lordcocoro2004/rcnn/ieeercnn_vgg16_1terranbase.h5')
model_zerg.load_weights('/home/lordcocoro2004/rcnn/ieeercnn_vgg16_1zergbase.h5')

/home/lordcocoro2004/miniconda3/envs/rcnn_2/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
z=0

for e,i in enumerate(sorted(os.listdir("/home/lordcocoro2004/SC_dataset/Dataset/video"))):
    if(not os. path. isfile('/home/lordcocoro2004/SC_dataset/Dataset/video_out/rec_'+i) and i.startswith("Comp")):
        st = time.time()
        print(i)
        z += 1
        img = cv2.imread(os.path.join("/home/lordcocoro2004/SC_dataset/Dataset/video",i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        
        count_g = 0
        count_m = 0
        count_p = 0
        count_t = 0
        count_z = 0
        
        for e,result in enumerate(ssresults):
            if e < 2000:
                x,y,w,h = result
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                
                out= model_gas.predict(img)                
                if out[0][0] > 0.70:
                    print(out[0][0],count_g,'gas')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (9, 247, 199), 1, cv2.LINE_AA)
                    count_g=count_g+1
                    
                out1= model_mineral.predict(img)
                if out1[0][0] > 0.70:
                    print(out1[0][0],count_g,'mineral')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 230, 255), 1, cv2.LINE_AA)
                    count_m=count_m+1
                    
                out2= model_protos.predict(img)
                if out2[0][0] > 0.70:
                    print(out2[0][0],count_g,'protos')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (255, 0, 0), 1, cv2.LINE_AA)
                    count_p=count_p+1
                    
                out3= model_terran.predict(img)
                if out3[0][0] > 0.70:
                    print(out3[0][0],count_g,'terran')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 14, 255), 1, cv2.LINE_AA)
                    count_t=count_t+1
                    
                out4= model_zerg.predict(img)
                if out4[0][0] > 0.70:
                    print(out[0][0],count_g,'zerg')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (28, 255, 0), 1, cv2.LINE_AA)
                    count_z=count_z+1
                #if count_g==7 or count_m==7 or count_p==7 or count_t==7 or count_z==7:
                #    break
                #if (time.time() - st) > 120 :
                #    break
        cv2.imwrite('/home/lordcocoro2004/SC_dataset/Dataset/video_out/rec_'+i, imout)
        print('Elapsed time = {}'.format(time.time() - st))
        #break

Comp 100.png
0.99293315 0 gas
0.99996364 1 gas
4.5288786e-09 2 zerg
0.9980312 2 gas
0.0 3 zerg
0.9474188 3 gas
Elapsed time = 391.4737694263458
Comp 101.png
0.9960865 0 gas
0.97669595 1 gas
0.99994135 2 gas
3.6921284e-09 3 zerg
0.9817821 3 gas
3.8133152e-07 4 zerg
0.7859838 4 gas
Elapsed time = 382.90605640411377
Comp 102.png
Elapsed time = 316.6731016635895
Comp 103.png
Elapsed time = 318.4577670097351
Comp 104.png
2.7587635e-10 0 zerg
0.9996075 0 gas
0.0 1 zerg
0.99519867 1 gas
0.99895895 2 gas
0.0 3 zerg
0.99986506 3 gas
0.9923309 4 gas
3.724541e-18 5 zerg
0.9129594 5 gas
Elapsed time = 395.7160301208496
Comp 105.png
Elapsed time = 329.5839638710022
Comp 106.png
Elapsed time = 345.4230682849884
Comp 107.png
1.4351431e-06 0 zerg
9.8999585e-14 0 zerg
Elapsed time = 278.40245723724365
Comp 108.png
1.4996791e-10 0 zerg
2.6304515e-06 0 zerg
Elapsed time = 307.2621703147888
Comp 109.png
1.0 0 protos
0.9976451 0 gas
1.0 1 protos
1.0 1 protos
0.9934876 1 gas
0.99999404 2 gas
1.0 3 protos
0.

In [10]:
z=0

for e,i in enumerate(sorted(os.listdir("/home/lordcocoro2004/SC_dataset/Dataset/video"))):
    if(i.startswith("Comp")):
        st = time.time()
        print(i)
        z += 1
        img = cv2.imread(os.path.join("/home/lordcocoro2004/SC_dataset/Dataset/video",i))
        ss.setBaseImage(img)
        #print(ss.setBaseImage(img))
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        
        count_g = 0
        count_m = 0
        count_p = 0
        count_t = 0
        count_z = 0
        
        print(ssresults)
        #for e,result in enumerate(ssresults):
            
            if e < 2000:
                print(result)
                x,y,w,h = result
                print(x,y,w,h)
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                
                out= model_gas.predict(img)                
                if out[0][0] > 0.70:
                    print(out[0][0],count_g,'gas')
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (9, 247, 199), 1, cv2.LINE_AA)
                    count_g=count_g+1
                    
                #if count_g==7 or count_m==7 or count_p==7 or count_t==7 or count_z==7:
                #    break
                #if (time.time() - st) > 120 :
                #    break
        #cv2.imwrite('/home/lordcocoro2004/SC_dataset/Dataset/video_out/rec_'+i, imout)
        print('Elapsed time = {}'.format(time.time() - st))
        #break

Comp 100.png
None
[[  1 164  58  27]
 [142  85  35  36]
 [  0   0 360 270]
 ...
 [  0   0 199  72]
 [  0   0 360 160]
 [  0  23 360 247]]
Elapsed time = 0.6368582248687744
Comp 101.png
None


KeyboardInterrupt: 